# SQL to Python
--------------------------------------
This notebook is for people with a solid undertsanding of SQL, trying to make a switch to Python for data science.

It shows how common SQL commands can be written in Python, and also shows how to start moving beyond just what you can do in SQL.

This notebook focuses on using Python for data analytics, with suggestions for further work later on.

I encourage you to take this notebook and run with it! Think of your own questions, and play around with the SQLite queries and Python code. You will learn much more trying to answer your own questions!

In [ ]:
# importing pandas, a "SQL-like" Python library with dataframe objects built in.
import pandas as pd

Dataset is from kaggle.
https://www.kaggle.com/carrie1/ecommerce-data. You don't have to download it though, as it is already in the repo folder.

In [ ]:
# Using pandas "read_csv' function to load the data.
data = pd.read_csv('ecommerce_data.csv', index_col=0, parse_dates=True)

Our data is e-commerce data from an e-commerce shop based in the UK. Most of the customers the company sells to are wholesalers.

In [ ]:
data.head()

## Making a local, server-less SQL database
[SQLite](https://www.sqlite.org/whentouse.html) is an easy-to-use SQL database system, that lets an entire database live in just one file on your computer, no server needed.

Here, we will make our own SQLite database in the repo folder, and populate with the same e-commerce data we just read into a pandas dataframe.
This way, we will be able to compare SQL commands (called to an actual SQL database) with their Python equvilents on the same data.

In [ ]:
import sqlite3, os, os.path

In [ ]:
# Seeing where our SQL database will be created.
os.getcwd()

In [ ]:
# Checking to see if a database with the same name already exists.
if os.path.isfile("E_commerce_data.db"):
    os.remove("E_commerce_data.db")

# Creating our new database, called E_commerce_data.db, and establishing a connection to it.
# We name this connection "conn". We will need to specify that connection in every SQL command we run,
# so that the queries are run on the correct database.
conn = sqlite3.connect("E_commerce_data.db")

In [ ]:
# formatting the invoice date
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], format="%m/%d/%Y %H:%M")

In [ ]:
# populating our new SQL database with the data from our dataframe.
data.to_sql('data', conn, if_exists='replace', index=False)

Taking an initial look at our data. 

We have ~40,000 records.

In [ ]:
data.describe()

Of this, about 36,000 of them are from the UK.

In [ ]:
data[data['Country']=='United Kingdom'].describe()

Describe will calculate summary statistics for all of the numeric columns.
It is also a good way to see if there are non-numeric columns that are being misclassified. 
"Customer ID", which is made up of numbers, was throught to be a numeric column when the CSV was read.

### Question 1: How many countries does are company sell to?

In [ ]:
len(data['Country'].unique())

# or:

len(set(data['Country']))

In [ ]:
pd.read_sql_query("""
SELECT COUNT(DISTINCT Country)
FROM data
""", conn)

### Question 2: What are our top 10 countries in terms of items sold?

In [ ]:
data.groupby(['Country']).Quantity.sum().sort_values(ascending=False).head(10)

In [ ]:
pd.read_sql_query("""
SELECT Country, SUM(Quantity)
FROM data
GROUP BY Country
ORDER BY SUM(Quantity) desc
LIMIT 10
""", conn)


### Question 3: How much money did we make on stickers in 2017?

In [ ]:
stickers = data[data['Description'].str.contains("STICKER SHEET")==True].reset_index()
stickers['Revenue'] = stickers['Quantity'] * stickers['UnitPrice']
stickers['Revenue'].sum()

In [ ]:
pd.read_sql_query("""
SELECT SUM(UnitPrice * Quantity)
FROM data
WHERE Description LIKE "%STICKER SHEET%"
""", conn)

### Question 4: How much money did we make in 2011 in the UK?

In [ ]:
uk_2011 = data.loc[(data['Country']=='United Kingdom') & (data['InvoiceDate'].dt.year==2011)] 
sum(uk_2011['UnitPrice'] * uk_2011['Quantity'])

In [ ]:
# SQLite doesn't have a year() function, so we have to get creative. 
# In the WHERE clause of our query, we have to format the date to just include its year component.
# You can read about date formats in SQLite here: https://www.tutorialspoint.com/sqlite/sqlite_date_time.html
pd.read_sql_query("""
SELECT SUM(UnitPrice * Quantity)
FROM data
WHERE Country = "United Kingdom" AND strftime('%Y', InvoiceDate) = '2011'
""", conn)

## What you can't do with SQL: more advanced analytics and visualizations

Starting: Building a Customer Segmentation

In [ ]:
data['Revenue'] = data['UnitPrice'] * data['Quantity']

In [ ]:
# removing canceled orders
data = data[data['Revenue'] > 0]

Step 1: group orders together by customer, aggregating the total number of orders they made, total quantity of all goods they've purchased, and total revenue gained.

In [ ]:
def create_customer_table(df):
    # Groups purchasing data by customer ID.
    
    # create number_orders df
    customer_df = df.groupby(by='CustomerID', as_index=False)['InvoiceNo'].count()
    
    # create total metrics: sum of float values
    for indicator in ['Quantity', 'Revenue']:
        new_col = df.groupby(by='CustomerID', as_index=False)[indicator].sum()
        customer_df = customer_df.merge(new_col, how='inner', right_on='CustomerID', left_on='CustomerID')
    
    customer_df.columns = ['CustomerID', 'NumberOrders', 'TotalQuantity', 'TotalRevenue']
    
    return customer_df

In [ ]:
customer_df = create_customer_table(data)
customer_df.tail()

Step 2: Calculating some additional metrics

In [ ]:
def calculate_customer_loyalty_metris(customer_df, df):
    # Step 1: create df with groupby for both the min and max invoice dates.
    first_order = df.groupby(by='CustomerID', as_index=False)['InvoiceDate'].min()
    first_order.columns = ['CustomerID', 'FirstOrder']
    
    last_order = df.groupby(by='CustomerID', as_index=False)['InvoiceDate'].max()
    last_order.columns = ['CustomerID', 'LastOrder']
    
    # Using a merge to combine the 2. This is just like a join in SQL.
    customer_metrics = first_order.merge(last_order, how='inner', right_on='CustomerID', left_on='CustomerID')
    
    # Adding column to use when calculating
    customer_metrics['Current'] = pd.to_datetime((df['InvoiceDate']).max())
    customer_metrics['CustomerTimeLength'] = 1 + (customer_metrics['Current']
                                                  - customer_metrics['FirstOrder']).astype('timedelta64[D]')

    customer_metrics['DaysSinceLastOrder'] = 1 + (customer_metrics['Current'] -
                                                 customer_metrics['LastOrder']).astype('timedelta64[D]')

    # dropping column we only used in calculating
    customer_metrics.drop(['Current', 'FirstOrder', 'LastOrder'], axis=1, inplace=True)
    
    # joining the table with the customer_df table
    customer_df = customer_df.merge(customer_metrics, how='inner', right_on='CustomerID', left_on='CustomerID')

    return customer_df

In [ ]:
customer_df = calculate_customer_loyalty_metris(customer_df, data)

# using iloc to select a random few rows to look at.
customer_df.iloc[3:10]

Now, we have more metrics to look into.

We can look at the average of each of these metrics, and define categories with them.

In [ ]:
customer_df.describe()

For most of the metrics, the mean far exceeds the median. This means that there are probably a few big customers (outliers) oredering a lot, and providing a lot more business than the average.

In [ ]:
def check_for_outliers(row, metric, cutoff):
    # helper function for creating customer segments
    if row[metric] > cutoff:
        return 'Yes'
    else:
        return 'No'

def create_customer_segments(customer_df):
    for metric in list(customer_df.columns)[1:]:
        cutoff = customer_df[metric].mean()
        customer_df[metric + 'Outlier'] = customer_df.apply(lambda x: check_for_outliers(x, metric, cutoff), axis=1)
    return customer_df

In [ ]:
customer_df = create_customer_segments(customer_df)

customer_df.iloc[10:15]

If a customer has a higher-than-average number of orders, total quantity, and total revenue, and has been a customer for longer than the average, we can call them a "Best Customer". If the customer's time since last order is greater than the average, we can assume they are an inactive customer. We will make a final column called "Customer Segments" where we place customers into one of 3 segments: a Best Customer, an Inactive Customer, and an Average Customer.

In [ ]:
def classify_customers(row):
    # helper function for creating customer classifications
    if (row['NumberOrdersOutlier'] == 'Yes' 
        and row['TotalQuantityOutlier'] == 'Yes' 
        and row['TotalRevenueOutlier'] == 'Yes' 
        and row['CustomerTimeLengthOutlier'] == 'Yes'):
        
        return 'Best Customer'
    
    elif row['DaysSinceLastOrderOutlier'] == 'Yes':
        return 'Inactive Customer'
    
    else:
        return 'Average Customer'

    
def create_customer_classifications(customer_df):
    customer_df['CustomerSegment'] = customer_df.apply(classify_customers, axis=1)
    return customer_df

In [ ]:
customer_df = create_customer_classifications(customer_df)

customer_df.iloc[0:5]

Now, we can do some visualizations with the data.

We will use [Seaborn](https://seaborn.pydata.org/index.html), a great Python library for making visualizations.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sorted_customer_df = customer_df.sort_values(by='TotalRevenue')
sorted_customer_df.reset_index(inplace=True)
sorted_customer_df['index'] = sorted_customer_df.index

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
col = sorted_customer_df['CustomerSegment'].map({'Best Customer':'r', 'Average Customer':'b', 'Inactive Customer':'black'})
sorted_customer_df.plot.scatter(x='index', y='NumberOrders', c=col);

In [ ]:
sns.set(style='ticks', font_scale = 1.2, rc={'lines.linewidth':0.0})
g = sns.relplot(x=sorted_customer_df.index,
                y="NumberOrders",
                hue="CustomerSegment",
                data=sorted_customer_df,
                height=12,
                facet_kws={'hue_kws':{'markers':'.', 'edgecolor':'None'}})
g;

In [ ]:
sns.axes_style()